            # Genetic Algorithm

Natural Selection based.

Initial Population -> Fitness Function -> Selection -> Crossover -> Mutation 

https://levelup.gitconnected.com/how-to-implement-a-traveling-salesman-problem-genetic-algorithm-in-python-ea32c7bef20f

In [1]:
import random

In [2]:
cities = [
    'F',  # "Fortaleza",
    'C',  # "Caucaia",
    'J',  # "Juazeiro do Norte",
    'M',  # "Maracanaú"
    'S'  # "SOBRAL"
]
costs = [
    [0, 10, 100, 50, 23],
    [10, 0, 20, 20, 12],
    [100, 20, 0, 10, 50],
    [50, 20, 10, 0, 17],
    [23, 12, 50, 17, 0]
] # (0)3412(0) = 50 + 17 + 12+ 20 +10    

# Idea: F -> C -> J -> M -> F

In [3]:
# Since we will always be back to initial state, doesn't seem smart to set start state inside the DNA  
bases = "1234"  # == C J M S

In [4]:
chromosome_example = "1342"  # = CMJ

In [5]:
def convert_chromosome(chromosome):
    if type(chromosome) is str:
        return list(chromosome)
    return ''.join(chromosome)

In [6]:
def fitness(chromosome: str, invert=False) -> int:
    cost = 0
    prev = 0
    for dna in chromosome:
        n = int(dna)
        cost += costs[prev][n]
        prev = n

    total_cost = (costs[0][int(chromosome[0])] + cost + 
                  costs[int(chromosome[len(chromosome) - 1])][0])
    return total_cost if invert is False else -total_cost

In [7]:
random_gen = random.Random(1)


def genesis(n: int) -> list[str]:
    pop = []
    for _ in range(n):
        dna = random_gen.sample(bases, len(bases))
        pop.append(convert_chromosome(dna))

    return pop

In [8]:
def progenitor_selection(population_set: list[str], choice_p=.2) -> list[tuple[str, str]]:
    progenitors = []
    valuable_progenitors = []
    for gene in population_set:
        v = fitness(gene)
        valuable_progenitors.append((gene, v))

    sorted_values = sorted(valuable_progenitors, key=lambda x: x[1])
    random_gen.randint(0, len(sorted_values) - 1)

    for index, (gene, fit) in enumerate(sorted_values):

        one = random_gen.random() > choice_p

        if index == len(sorted_values) - 1:
            progenitors.append((gene, 0 if one else random_gen.randint(index, len(sorted_values) - 1)))
            continue

        progenitors.append(
            (gene, sorted_values[(index + 1) if one else random_gen.randint(index, len(sorted_values) - 1)])
        )

    return progenitors

In [9]:
def mate_progenitors(progenitors: tuple[str, str], mate=.5):
    dad, mom = progenitors
    finish = int(mate * len(dad))
    son = dad[:finish]

    to_create = len(dad) - finish
    mom_copy = mom[:]
    mom_copy.reverse()
    for gene in mom_copy:
        if gene not in son:
            son[finish] = gene
            finish += 1

        if finish == to_create:
            break

    return son


In [10]:
def mutate(cromo: str, mp=.1):
    chromo2 = convert_chromosome(cromo)
    while random_gen.random() > mp:
        a = random_gen.randint(0, len(chromo2) - 1)
        b = random_gen.randint(0, len(chromo2) - 1)
        chromo2[a], chromo2[b] = chromo2[b], chromo2[a]

    return convert_chromosome(chromo2)

In [14]:
from heapq import heappush, heappop

heap = []


parents = genesis(10)

for chromo in parents:
    v = fitness(chromo)
    heappush(heap, (v, chromo))
    
selection = progenitor_selection(parents)

parents = []
for mate in selection:
    son = mate_progenitors(mate)
    son = mutate(son)
    parents.append(son)

for chromo in parents:
    v = fitness(chromo)
    heappush(heap, (v, chromo))

# for chromo in range(100):
#     # if i%100==0: print(i, fitnes_list.min(), fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
#     
#     
#     heappush(heap, )    

KeyboardInterrupt: 